# Google Analytics Customer Revenue

## Import Libraries

In [1]:
import sys
import os 

import numpy as np # Arrays
import pandas as pd # Series and Dataframes

import matplotlib.pyplot as plt 
import seaborn as sns # Advanced Plotting 


pd.options.display.max_rows = 100
plt.style.use('ggplot') # Beautify Plots 

In [2]:
import random
import json 
from pandas.io.json import json_normalize
from scipy import stats
from datetime import datetime
from sklearn import model_selection
import xgboost as xgb
#import lightgbm as lgb

In [3]:
RANDOM_STATE = 42

## Import Data 

In [4]:
## Función de lectura 
columns = ['device', 'geoNetwork', 'totals'] # Columns with json format

p = 0.1 # Fraction of data to use

def json_read(df):
    data_frame = file_dir + df
    
    df = pd.read_csv(data_frame, 
                     converters={column: json.loads for column in columns}, 
                     dtype={'fullVisitorId': 'str'}, 
                     skiprows=lambda i: i>0 and random.random() > p)
    
    for column in columns: 
        column_as_df = json_normalize(df[column]) 
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns] 
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
        
    print(f"Loaded {os.path.basename(data_frame)}. Shape: {df.shape}")
    return df

In [5]:
file_dir = ''
file_name = 'GACR_def.csv'

In [6]:
%%time
df = json_read(file_name)

<ipython-input-4-b09283f7f4e4>:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  column_as_df = json_normalize(df[column])
Loaded GACR_def.csv. Shape: (90594, 42)
Wall time: 13.5 s


## EDA - Exploratory Data Analysis

In [7]:
df.info(verbose=False)
# info: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90594 entries, 0 to 90593
Columns: 42 entries, Unnamed: 0 to totals.transactionRevenue
dtypes: bool(1), int64(5), object(36)
memory usage: 28.4+ MB


In [15]:
df.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,...,geoNetwork.networkDomain,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.networkLocation,totals.visits,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.transactionRevenue
0,Organic Search,20160902,1905672039242460897,1905672039242460897_1472817241,Not Socially Engaged,1472817241,1,1472817241,Chrome,not available in demo dataset,...,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,NaN
1,Organic Search,20160902,4445454811831400414,4445454811831400414_1472805784,Not Socially Engaged,1472805784,1,1472805784,Internet Explorer,not available in demo dataset,...,spar.at,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,NaN
2,Organic Search,20160902,9499785259412240342,9499785259412240342_1472812272,Not Socially Engaged,1472812272,1,1472812272,Firefox,not available in demo dataset,...,chello.nl,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,NaN
3,Organic Search,20160902,357659889600827884,357659889600827884_1472839882,Not Socially Engaged,1472839882,1,1472839882,Safari,not available in demo dataset,...,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,NaN
4,Organic Search,20160902,5300348437898869056,5300348437898869056_1472810280,Not Socially Engaged,1472810280,1,1472810280,Chrome,not available in demo dataset,...,proxad.net,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,NaN


In [14]:
df.drop('Unnamed: 0', axis='columns', inplace=True)

In [16]:
# me quedo con el sessionId como identificador de fila
df.set_index('sessionId', inplace=True)

In [17]:
df.head(2)

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,device.browserSize,...,geoNetwork.networkDomain,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.networkLocation,totals.visits,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.transactionRevenue
sessionId,,,,,,,,,,,,,,,,,,,,,
1905672039242460897_1472817241,Organic Search,20160902,1905672039242460897,Not Socially Engaged,1472817241,1,1472817241,Chrome,not available in demo dataset,not available in demo dataset,...,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,NaN
4445454811831400414_1472805784,Organic Search,20160902,4445454811831400414,Not Socially Engaged,1472805784,1,1472805784,Internet Explorer,not available in demo dataset,not available in demo dataset,...,spar.at,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,NaN


In [18]:
### Borramos los otros Ids, me he quedato con sessionId
df_ids = ['fullVisitorId', 'visitId']
df.drop(df_ids, axis=1, inplace=True)

In [22]:
df.head(4).T

sessionId,1905672039242460897_1472817241,4445454811831400414_1472805784,9499785259412240342_1472812272,357659889600827884_1472839882
channelGrouping,Organic Search,Organic Search,Organic Search,Organic Search
date,20160902,20160902,20160902,20160902
socialEngagementType,Not Socially Engaged,Not Socially Engaged,Not Socially Engaged,Not Socially Engaged
visitNumber,1,1,1,1
visitStartTime,1472817241,1472805784,1472812272,1472839882
device.browser,Chrome,Internet Explorer,Firefox,Safari
device.browserVersion,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset
device.browserSize,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset
device.operatingSystem,Windows,Windows,Windows,iOS
device.operatingSystemVersion,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset


In [23]:
# sin ningun atributo "include" me da las numericas
# de esa manera lo mismo: me da las numericas
# tenemos solo 3 variables numericas -> pero dos ni lo son: 2 son datetime
df.describe(include=np.number).T

,count,mean,std,min,25%,50%,75%,max
date,90594.0,2.016589e+07,4.697240e+03,2.016080e+07,2.016103e+07,2.017011e+07,2.017042e+07,2.017080e+07
visitNumber,90594.0,2.268285e+00,9.356026e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.840000e+02
visitStartTime,90594.0,1.485003e+09,9.012520e+06,1.470036e+09,1.477556e+09,1.483953e+09,1.492710e+09,1.501657e+09


In [24]:
# aqui miramos las CATEGORICAS
df.describe(exclude=np.number).T

,count,unique,top,freq
channelGrouping,90594,8,Organic Search,38344
socialEngagementType,90594,1,Not Socially Engaged,90594
device.browser,90594,32,Chrome,62227
device.browserVersion,90594,1,not available in demo dataset,90594
device.browserSize,90594,1,not available in demo dataset,90594
device.operatingSystem,90594,16,Windows,35150
device.operatingSystemVersion,90594,1,not available in demo dataset,90594
device.isMobile,90594,2,False,66713
device.mobileDeviceBranding,90594,1,not available in demo dataset,90594
device.mobileDeviceModel,90594,1,not available in demo dataset,90594


In [25]:
# cuantos distintos por cada columna
df.nunique()

channelGrouping                         8
date                                  366
socialEngagementType                    1
visitNumber                           209
visitStartTime                      90417
device.browser                         32
device.browserVersion                   1
device.browserSize                      1
device.operatingSystem                 16
device.operatingSystemVersion           1
device.isMobile                         2
device.mobileDeviceBranding             1
device.mobileDeviceModel                1
device.mobileInputSelector              1
device.mobileDeviceInfo                 1
device.mobileDeviceMarketingName        1
device.flashVersion                     1
device.language                         1
device.screenColors                     1
device.screenResolution                 1
device.deviceCategory                   3
geoNetwork.continent                    6
geoNetwork.subContinent                23
geoNetwork.country                

In [ ]:
# constant_columns = []
# for column in df.columns: # para cada columna (string) hacer algo
#        constant_columns.append(column)
#    if df[column].nunique() == 1:

In [26]:
constant_columns = [column for column in df.columns if df[column].nunique() == 1]

In [27]:
# la lista de las columnas que tienen valor unico (osea presenta NINGUNA VARIANZA) -> se van a borrar
constant_columns

['socialEngagementType',
 'device.browserVersion',
 'device.browserSize',
 'device.operatingSystemVersion',
 'device.mobileDeviceBranding',
 'device.mobileDeviceModel',
 'device.mobileInputSelector',
 'device.mobileDeviceInfo',
 'device.mobileDeviceMarketingName',
 'device.flashVersion',
 'device.language',
 'device.screenColors',
 'device.screenResolution',
 'geoNetwork.cityId',
 'geoNetwork.latitude',
 'geoNetwork.longitude',
 'geoNetwork.networkLocation',
 'totals.visits',
 'totals.bounces',
 'totals.newVisits']

In [28]:
# 1:38:00
df.drop(constant_columns, axis=1, inplace=True)

In [ ]:
df.info(verbose=False)

In [ ]:
df.head(2).T

In [ ]:
df_description = df.describe(include='all').T

In [ ]:
df_description.to_excel('df_description.xlsx')

#### Target Analysis 

In [ ]:
TARGET = 'totals.transactionRevenue'

In [ ]:
df[TARGET].describe()

In [ ]:
df[TARGET].head()

In [ ]:
df[df[TARGET].notnull()][TARGET].head(20)

In [ ]:
df[TARGET] = df[TARGET].astype(float)

In [ ]:
df[TARGET].describe()

In [ ]:
df[TARGET].fillna(0.0, inplace=True)

In [ ]:
df[TARGET].describe()

In [ ]:
df[TARGET] = df[TARGET] / 1000000

In [ ]:
df[TARGET].describe()

In [ ]:
# describe sobre aquella parte de la variable > 0 (osea, que hai compras)
# SUBSET del DF
df[df[TARGET]>0][TARGET].describe()

In [ ]:
df['visitWithTransaction'] = (df[TARGET] > 0).astype(int)

In [ ]:
df.head().T

In [ ]:
df['visitWithTransaction'].sum()

In [ ]:
df['visitWithTransaction'].mean() * 100

In [ ]:
print('De las {} visitas, hay {} con compras, lo que significa que el {} por ciento de las visitas tienen compra'.format(
    df['visitWithTransaction'].count(),
    df['visitWithTransaction'].sum(),
    df['visitWithTransaction'].mean() * 100
))

In [ ]:
# Distribución de la variable target al completo
plt.figure(figsize=(15, 5))
sns.distplot(
    df[TARGET]
)

In [ ]:
# Distribución de la variable target al completo
plt.figure(figsize=(15, 5))
sns.distplot(
    df[df['visitWithTransaction'] == 1][TARGET],
    fit = stats.norm
)

In [ ]:
## normalizar la variable TARGET con su version logaritmica ()

In [ ]:
df['totals.transactionRevenue'].head()

In [ ]:
df['totals.transactionRevenueLN'] = df['totals.transactionRevenue'].apply(lambda x: np.log1p(x))

In [ ]:
TARGET_LN = 'totals.transactionRevenueLN'

In [ ]:
# Distribución de la variable target transformada al completo
plt.figure(figsize=(15, 5))
sns.distplot(
    df[df['visitWithTransaction'] == 1][TARGET_LN],
    fit = stats.norm
)

### Device - Device Browser

In [ ]:
df['device.browser'].describe()

In [ ]:
top_browsers = df['device.browser'].value_counts().head(5)
# muchos valores tienen tan pocas observaciones que el decision three nunca hara un corte
# en muchos casos es mejor agrupar las etiquetas en una nueva categoria "Otro_navigator"
# (en todo casos haremos un OHE, pero reduciendo el numero de valores de la variables)

# setOthers -> etiquetas y numero de observaciones
# la que tienen pocas observaciones, no es importante hacer OHE (porque el DecisionThree no va a tener cuenta)
# asi que, agrupamos en una nueva categoria (Others) las que tienen pocas observaciones

In [ ]:
top_browsers

In [ ]:
# saco el indice y lo pongo en una lista (de las etiquetas que para mi son mas importantes -> 3,4,5)
top_browsers_list = top_browsers.index.to_list()

In [ ]:
top_browsers_list

In [ ]:
# pongo una nueva categoria -> Others
top_browsers_list.append('Others')

In [ ]:
top_browsers_list

In [ ]:
# de mi variable, la voy a pasar CATEGORICA con las 3 etiquetas
df['device.browser'] = pd.Categorical(df['device.browser'], categories=top_browsers_list)

In [ ]:
# ahora ocure que tenemos las etiquetas, mas una nueva etiqueta
df['device.browser'].value_counts(dropna=False)

In [ ]:
# asigno los NaN a la nueva columna "Others"
df['device.browser'].fillna('Others', inplace=True)

In [ ]:
df['device.browser'].describe()

In [ ]:
df['device.browser'].value_counts(dropna=False)

In [ ]:
df['device.browser'].value_counts(normalize=True, dropna=False)

In [ ]:
# analisi de como se corelacione una variable con el target
df[df['visitWithTransaction'] == 1].pivot_table(index='device.browser', values=TARGET_LN, aggfunc=[len, np.mean])

In [ ]:
_results_df = df[df['visitWithTransaction'] == 1].pivot_table(index='device.browser', values=TARGET_LN, aggfunc=[len, np.mean])

In [ ]:
# cambio el nombre de las columnas del dataframe que acabo e sacarme
_results_df.columns = ['transactions', 'mean_revenue_ln']

In [ ]:
_results_df.head()

In [ ]:
# añado una columna con numero de filas
_results_df['n_rows'] = df['device.browser'].value_counts()

In [ ]:
_results_df.head()

In [ ]:
# añado una columna con numero de filas en % (con normalize)
# me da de todas las filas, que porcentaje son de chrome, safari, firefox y others
# esta columna me dice que % vienen de cual browser
_results_df['pct_rows'] = df['device.browser'].value_counts(normalize=True)

In [ ]:
_results_df.head()

In [ ]:
# 
_results_df['pct_transactions'] = _results_df['transactions'] / _results_df['n_rows']

In [ ]:
_results_df.head()

In [ ]:
# reordenar el DF
_results_df = _results_df[['n_rows', 'pct_rows', 'transactions', 'pct_transactions', 'mean_revenue_ln']]

In [ ]:
_results_df

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot(
    df['device.browser']
)

<p style="font-size:2em; color:yellow;">----------------- ESTOY AQUI -------------------------</p>

In [ ]:
def plot_cat_values(dataframe, column, target_column, _target):
    plt.figure(figsize=(15,8))
    # GRAFICO: COUNT
    #en una única grafica de 2 filas de graficos x 1 columna en el slot 1
    ax1 = plt.subplot(2,1,1)
    ## Graficamos el conteo de cada uno de los valores
    ax1 = sns.countplot(
        data=dataframe[column],
        order = list(dataframe[column].unique())
    )

    # GRAFICO: DISTRIBUCION DE LA COLUMNA ENTRE LOS QUE COMPRAN
    ax2 = plt.subplot(2,1,2) #en una única grafica de 2 filas de graficos x 1 columna en el slot 2
    ## Graficamos la distribución del target sólo para aquellos casos con target > 0 (para que no se chafe el BP)
    # si tenemos variable muy sesgada, los percentiles seran el mismo numero (mejor cojer la parte que cumple el target)
    ax2 = sns.boxenplot(
        data = dataframe[dataframe[target_column] > 0], 
        x = column,
        y = _target,
        order = list(dataframe[column].unique())
    )
    plt.show()


In [ ]:
plot_cat_values(df, 'device.browser', TARGET_LN)

In [ ]:
# hacer analisis de la variable en respecto al target
# valor medio, algun conteo mas nos DAN LA CORELACION de el TARGET con la variable que estamos tratando
def explore_cat_values(dataframe, column, column_target_1, target):
    # pivot table para sacar info estadisticas, como se corelacione una variable con el target
    _results_df = dataframe[dataframe[column_target_1] > 1].pivot_table(index=column, values=target, aggfunc=[len, np.mean])
    # renombrar las columnas
    _results_df.columns = ['transactions', 'mean_revenue_ln']
    # value count con NULOS, que asigno a nueva columna
    _results_df['n_rows'] = dataframe[column].value_counts(dropna=False)
    # nueva columna con porcentaje
    _results_df['pct_rows'] = dataframe[column].value_counts(normalize=True, dropna=False)
    # nueva columna con porcentaje
    _results_df['pct_transactions'] = _results_df['transactions'] / _results_df['n_rows']
    # ordenar el DF
    _results_df = _results_df[['n_rows', 'pct_rows', 'transaction', 'pct_transaction', 'mean_revenue_ln']]
    
    return _results_df

In [ ]:
# este plot nos da la misma info que la tabla de hariba
plt.figure(figsize=(15,5))
sns.boxenplot(
    # cogemos el Df de los solo que segatsan algo (que han comprado)
    data = df[df['visitWithTransaction'] == 1], 
    x = 'device.browser',
    y = TARGET_LN
)

In [ ]:
explore_cat_values(df, 'device.browser', TARGET_LN)

In [ ]:
# M
def setOthers(dataframe, column, num_values):
    # quedamos con la top etiquetas que queremos (las primeras 3 en este ejemplo)
    top_categories = dataframe[column].value_counts().head(num_values)
    # generamos la lista con las 3 etiquetas con la que nos quedamos
    top_categories_list = top_categories.index.to_list()
    # añado una columna Others a la lista (donde pondre los NaN)
    top_categories_list.append('Others')
    # fuerzo que sea categorica -> (numero de etiquetas finito y ademas son predefinida)
    # el pd.categorical busca si existen etiquetas Others, todos los que tenian otro nombre, lo deja nulo
    dataframe[column] = pd.Categorical(dataframe[column], categories=top_categories_list)
    # devuelve el DF, con los NULOS puesto en la columna others
    return dataframe[column].fillna('Others')

### Sistema Operativo

In [ ]:
df['device.operatingSystem'].value_counts()

In [ ]:
df['device.operatingSystem'] = setOthers(df, 'device.operatingSystem', 6)

In [ ]:
df['device.operatingSystem'].value_counts()

In [ ]:
plot_cat_values(df, 'device.operatingSystem', TARGET_LN)

In [ ]:
explore_cat_values(df, 'device.operatingSystem', TARGET_LN)

### Device - Is Mobile

In [ ]:
explore_cat_values(df, 'device.isMobile', TARGET_LN)

Se observa que en Desktop se hace compra en un mayor porcentaje de ocasiones y el importe es más alto

In [ ]:
df['device.isMobile'].dtype

In [ ]:
df['device.isMobile'] = df['device.isMobile'].astype(int)

In [ ]:
df['device.isMobile'].dtype

### Device - Category 

In [ ]:
explore_cat_values(df, 'device.deviceCategory', TARGET_LN)

In [ ]:
pd.crosstab(df['device.isMobile'], df['device.deviceCategory'])

### ChannelGrouping

In [ ]:
explore_cat_values(df, 'channelGrouping', TARGET_LN)

In [ ]:
plot_cat_values(df, 'channelGrouping', TARGET_LN)

### Date

In [ ]:
df['date'].describe()

In [ ]:
df['date'].head()

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

In [ ]:
df['date'].head()

In [ ]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['monthDay'] = df['date'].dt.day
df['weekDay'] = df['date'].dt.weekday
df['quarter'] = df['date'].dt.quarter
df['week'] = df['date'].dt.week

In [ ]:
df.head(5).T

In [ ]:
df.drop('date', axis=1, inplace=True)

### Visit Start Time

In [ ]:
df['visitStartTime'].describe()

In [ ]:
df['visitStartTime'].head()

In [ ]:
df['visitHour'] = df['visitStartTime'].apply(lambda x: datetime.fromtimestamp(x).hour)

In [ ]:
df.head().T

In [ ]:
df.drop('visitStartTime', axis=1, inplace=True)

### Visit Number

In [ ]:
df['visitNumber'].head()

In [ ]:
df['visitNumber'].describe()

In [ ]:
plt.figure(figsize=(15,5))
sns.regplot(
    data=df[df['visitWithTransaction'] == 1],
    x='visitNumber',
    y=TARGET_LN
)

In [ ]:
df[['visitNumber', TARGET_LN]].head()

In [ ]:
df[['visitNumber', TARGET_LN]].corr()

In [ ]:
df['visitNumberLN'] = df['visitNumber'].apply(lambda x: np.log1p(x))

In [ ]:
plt.figure(figsize=(15,5))
sns.regplot(
    data=df[df['visitWithTransaction'] == 1],
    x='visitNumberLN',
    y=TARGET_LN
)

In [ ]:
df[['visitNumberLN', TARGET_LN]].corr()

### Hits

In [ ]:
df['totals.hits'].head(5)

In [ ]:
df['totals.hits'].describe()

In [ ]:
df['totals.hits'] = df['totals.hits'].astype(int)

In [ ]:
df['totals.hits'].describe()

In [ ]:
plt.figure(figsize=(15,5))
sns.regplot(
    data=df[df['visitWithTransaction'] == 1],
    x='totals.hits',
    y=TARGET_LN
)

In [ ]:
df['totals.hitsLN'] = df['totals.hits'].apply(lambda x: np.log1p(x))

In [ ]:
plt.figure(figsize=(15,5))
sns.regplot(
    data=df[df['visitWithTransaction'] == 1],
    x='totals.hitsLN',
    y=TARGET_LN
)

In [ ]:
df[['totals.hitsLN', TARGET_LN]].corr()

In [ ]:
df.pivot_table(index='totals.hits', values='visitWithTransaction', aggfunc=[len, np.sum, np.mean]).head(30)

### PageViews

In [ ]:
df['totals.pageviews'].head(5)

In [ ]:
df['totals.pageviews'].fillna(0, inplace=True)

In [ ]:
df['totals.pageviews'] = df['totals.pageviews'].astype(int)

In [ ]:
df['totals.pageviews'].describe()

In [ ]:
plt.figure(figsize=(15,5))
sns.regplot(
    data=df[df['visitWithTransaction'] == 1],
    x='totals.pageviews',
    y=TARGET_LN
)

In [ ]:
df['totals.pageviewsLN'] = df['totals.pageviews'].apply(lambda x: np.log1p(x))

In [ ]:
plt.figure(figsize=(15,5))
sns.regplot(
    data=df[df['visitWithTransaction'] == 1],
    x='totals.pageviewsLN',
    y=TARGET_LN
)

Podemos analizar la correlación entre PV y Hits 

In [ ]:
df[['totals.pageviews', 'totals.hits']].corr()

### GeoNetwork - Continent 

In [ ]:
explore_cat_values(df, 'geoNetwork.continent', TARGET_LN)

In [ ]:
plot_cat_values(df, 'geoNetwork.continent', TARGET_LN)

### GeoNetwork - SubContinent 

In [ ]:
explore_cat_values(df, 'geoNetwork.subContinent', TARGET_LN)

### GeoNetwork - Country

In [ ]:
results_by_country = explore_cat_values(df, 'geoNetwork.country', TARGET_LN)

In [ ]:
results_by_country

In [ ]:
last_countries = results_by_country[results_by_country['n_rows'] > 500].sort_values(by='mean_revenue_ln').head().index.to_list()

In [ ]:
last_countries

In [ ]:
first_countries = results_by_country[results_by_country['n_rows'] > 500].sort_values(by='mean_revenue_ln').tail().index.to_list()

In [ ]:
first_countries

In [ ]:
country_list = last_countries + first_countries

In [ ]:
country_list

In [ ]:
#
def setOthersPataNegra(dataframe, column, target_column, num_rows_min, top_n):
    #
    results_by_category = explore_cat_values(dataframe, column, target_column)
    # saco los primero 5 (meno compran)
    last_categories = results_by_category[results_by_category['n_rows'] > num_rows_min].sort_values(by='mean_revenue_ln').head(top_n).index.to_list()
    # saco las ultimas 5 (mas compran)
    first_categories = results_by_category[results_by_category['n_rows'] > num_rows_min].sort_values(by='mean_revenue_ln').tail(top_n).index.to_list()
    top_categories_list = first_categories + last_categories
    top_categories_list = set(top_categories_list)
    top_categories_list.append('Others')
    # convierte la variable, de todas las etiquetas que tenia a solo las etiquetas del top_cat_list + Others
    dataframe[column] = pd.Categorical(dataframe[column], categories=top_categories_list)
    return dataframe[column].fillna('Others')

In [ ]:
df['geoNetwork.country'] = setOthersPataNegra(df, 'geoNetwork.country', TARGET_LN, 500, 5)

In [ ]:
df['geoNetwork.country'].value_counts()

### geoNetwork - City

In [ ]:
explore_cat_values(df, 'geoNetwork.city', TARGET_LN).sort_values(by='mean_revenue_ln')

In [ ]:
df['geoNetwork.city'] = setOthersPataNegra(df, 'geoNetwork.city', TARGET_LN, 100, 10)

In [ ]:
df.info()

### GeoNetwork - Metro

In [ ]:
df['geoNetwork.metro'].head(20)

In [ ]:
df['geoNetwork.metro'].describe()

In [ ]:
explore_cat_values(df, 'geoNetwork.metro', TARGET_LN)

In [ ]:
df.drop('geoNetwork.metro', axis=1, inplace=True)

### GeoNetwork - Network Domain 

In [ ]:
df['geoNetwork.networkDomain'].head()

In [ ]:
df['geoNetwork.networkDomain'].describe()

In [ ]:
df['geoNetwork.networkDomain'].value_counts().head(30)

In [ ]:
df['network_net'] = df['geoNetwork.networkDomain'].str.contains('.net', case=False).astype(int)

In [ ]:
df['network_com'] = df['geoNetwork.networkDomain'].str.contains('.com', case=False).astype(int)

In [ ]:
results_by_network = explore_cat_values(df, 'geoNetwork.networkDomain', TARGET_LN)

In [ ]:
results_by_network[results_by_network['n_rows'] > 500]

In [ ]:
df['geoNetwork.networkDomain'] = setOthers(df, 'geoNetwork.networkDomain', 10)

### GeoNetwork - Region

In [ ]:
df['geoNetwork.region'].head(10)

In [ ]:
df['geoNetwork.region'].describe()

In [ ]:
df.drop('geoNetwork.region', axis=1, inplace=True)

In [ ]:
df.info()

In [ ]:
df.describe(include=np.number).T

In [ ]:
df.describe(exclude=np.number).T

## Final DataFrame

In [ ]:
df.describe(include=np.number).T

In [ ]:
df.describe(exclude=np.number).T

In [ ]:
def generate_dummies(dataframe, column_name):
    _dummy_dataset = pd.get_dummies(dataframe[column_name], prefix=column_name)
    dataframe = pd.concat([dataframe, _dummy_dataset], axis=1)
    return dataframe.drop(column_name, axis=1)

In [ ]:
for column in df.select_dtypes(exclude=np.number).columns:
    df = generate_dummies(df, column)

In [ ]:
df.info()

In [ ]:
df.describe(include=np.number).T.head(20)

## Modelling 

In [ ]:
target_linked_features = ['totals.transactionRevenue', 'visitWithTransaction']

In [ ]:
df.drop(target_linked_features, axis=1, inplace=True)

### Estrategia de validación

In [ ]:
df.pivot_table(index=['year', 'month'], values=TARGET_LN, aggfunc=len)

In [ ]:
df_val = df[df['year'] * 100 + df['month'] >= 201706]
df_dev = df[df['year'] * 100 + df['month'] < 201706]

In [ ]:
df_val.info()

In [ ]:
df_dev.info()

In [ ]:
df_val_X = df_val.drop(TARGET_LN, axis=1)
df_val_y = df_val[[TARGET_LN]]
df_dev_X = df_dev.drop(TARGET_LN, axis=1)
df_dev_y = df_dev[[TARGET_LN]]

In [ ]:
df_val_X.info(verbose=False)

In [ ]:
df_val_y.info(verbose=False)

In [ ]:
df_dev_X.info(verbose=False)

In [ ]:
df_dev_y.info(verbose=False)

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    df_dev_X,
    df_dev_y,
    random_state = RANDOM_STATE,
    test_size = 0.3
)

In [ ]:
X_train.info(verbose=False)

In [ ]:
X_test.info(verbose=False)

In [ ]:
y_train.info(verbose=False)

In [ ]:
y_test.info(verbose=False)

### Prueba de algoritmos

In [ ]:
first_model = xgb.XGBRegressor(random_state=42, n_estimators=100, max_depth=4)

In [ ]:
%%time
first_model.fit(X_train, y_train)

In [ ]:
test_predictions = pd.DataFrame(first_model.predict(X_test), columns=['Prediction'], index=X_test.index)

In [ ]:
test_predictions.head(10)

In [ ]:
y_test.head(10)

In [ ]:
results_df = y_test.join(test_predictions)

In [ ]:
results_df.sort_values(by='totals.transactionRevenueLN', ascending=False).head(19)

In [ ]:
results_df.columns = ['Target', 'Prediction']

In [ ]:
results_df['error'] = results_df['Target'] - results_df['Prediction']

In [ ]:
results_df.head()

In [ ]:
results_df['squared_error'] = results_df['error'] ** 2

In [ ]:
results_df['rooted_squared_error'] = np.sqrt(results_df['squared_error'])

In [ ]:
results_df.sort_values(by='Target', ascending=False).head()

In [ ]:
mse = results_df['squared_error'].mean()
rmse = results_df['rooted_squared_error'].mean()

In [ ]:
print('MSE: {} - RMSE: {}'.format(mse, rmse))

In [ ]:
plt.figure(figsize=(15,5))
sns.distplot(
    results_df['rooted_squared_error']
)

In [ ]:
plt.figure(figsize=(15,5))
sns.distplot(
    results_df[results_df['Target'] > 0]['rooted_squared_error'],
    fit=stats.norm
)

In [ ]:
plt.figure(figsize=(15,5))
sns.distplot(
    results_df[results_df['Target'] > 0]['Target'],
    fit=stats.norm
)

#### Vamos a rebalancear el DF

In [ ]:
df_dev_zero = df_dev[df_dev[TARGET_LN] == 0]

In [ ]:
df_dev_nonzero = df_dev[df_dev[TARGET_LN] > 0]

In [ ]:
df_dev_zero.info(verbose=False)

In [ ]:
df_dev_nonzero.info(verbose=False)

In [ ]:
n_nonzeros = len(df_dev_nonzero) * 3

In [ ]:
n_nonzeros

In [ ]:
df_dev_zero_sample = df_dev_zero.sample(n = n_nonzeros, random_state = RANDOM_STATE)

In [ ]:
df_dev_zero_sample.info()

In [ ]:
df_dev_sample = pd.concat([df_dev_nonzero, df_dev_zero_sample])

In [ ]:
df_dev_sample.info(verbose=False)

In [ ]:
df_dev_sample_X = df_dev_sample.drop(TARGET_LN, axis=1)
df_dev_sample_y = df_dev_sample[[TARGET_LN]]

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    df_dev_sample_X,
    df_dev_sample_y,
    random_state = RANDOM_STATE,
    test_size = 0.3
)

In [ ]:
X_train.info(verbose=False)

In [ ]:
y_train.info(verbose=False)

In [ ]:
X_test.info(verbose=False)

In [ ]:
y_test.info(verbose=False)

In [ ]:
resampled_model = xgb.XGBRegressor(max_depth=4, random_state=RANDOM_STATE, n_estimators=100)

In [ ]:
%%time
resampled_model.fit(X_train, y_train)

In [ ]:
test_predictions = pd.DataFrame(resampled_model.predict(X_test), index=X_test.index, columns=['Prediction'])

In [ ]:
results_df = y_test.join(test_predictions)

In [ ]:
results_df.head()

In [ ]:
results_df.columns = ['Target', 'Prediction']

In [ ]:
results_df['error'] = results_df['Target'] - results_df['Prediction']
results_df['squared_error'] = results_df['error'] ** 2
results_df['rooted_squared_error'] = np.sqrt(results_df['squared_error'])
mse = results_df['squared_error'].mean()
rmse = results_df['rooted_squared_error'].mean()

In [ ]:
print('MSE: {} - RMSE: {}'.format(mse, rmse))

In [ ]:
plt.figure(figsize=(15,5))
sns.distplot(
    results_df[results_df['Target'] > 0]['rooted_squared_error'],
    fit=stats.norm
)

In [ ]:
plt.figure(figsize=(15,5))
sns.distplot(
    results_df[results_df['Target'] > 0]['Target'],
    fit=stats.norm
)

In [ ]:
results_df.sort_values(by='Target', ascending=False).head(10)

#### Cerramos el modelo con el DF de validación

In [ ]:
val_predictions = pd.DataFrame(resampled_model.predict(df_val_X), index=df_val_X.index, columns=['Prediction'])

In [ ]:
val_results_df = df_val_y.join(val_predictions)

In [ ]:
val_results_df.head()

In [ ]:
val_results_df.columns = ['Target', 'Prediction']

In [ ]:
val_results_df['error'] = val_results_df['Target'] - val_results_df['Prediction']
val_results_df['squared_error'] = val_results_df['error'] ** 2
val_results_df['rooted_squared_error'] = np.sqrt(val_results_df['squared_error'])
mse = val_results_df['squared_error'].mean()
rmse = val_results_df['rooted_squared_error'].mean()

In [ ]:
print('MSE: {} - RMSE: {}'.format(mse, rmse))

In [ ]:
plt.figure(figsize=(15,5))
sns.distplot(
    val_results_df[val_results_df['Target'] > 0]['rooted_squared_error'],
    fit=stats.norm
)